In [ ]:
import sys
import os
import SSGATE as ssgate
import scanpy as sc

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# This tutorial is used to illustrate the steps of multi-omics integration of the SCS_MT dataset.
# Import transcriptome and proteome data in h5ad format
adata_sp = sc.read_h5ad('SCS_MT_SP_bin50_filtered_raw.h5ad')
adata_st = sc.read_h5ad('SCS_MT_ST_bin50_filtered_raw.h5ad')

In [ ]:
# The above data is not filtered or standardized by default, 
# and is preprocessed using the preprocessing function of SSGATE.
adata_st, adata_sp = ssgate.preprocess_cluster(adata_st, adata_sp, res_st = 0.5, res_sp = 0.2, show_fig = True, figsize = (6,3))

In [ ]:
# Neighbor network construction, 
# network pruning, 
# and neighbor network statistics based on transcriptome data
adata_st = ssgate.Cal_Nbrs_Net(adata_st, feat = "X_pca", k_cutoff = 15, model = "KNN")
adata_st = ssgate.prune_net(adata_st)
ssgate.Stats_Nbrs_Net(adata_st)

In [ ]:
# Proteomic data are processed similarly to transcriptomic data
adata_sp = ssgate.Cal_Nbrs_Net(adata_sp, feat = "X_pca", k_cutoff = 15, model = "KNN")
adata_sp = ssgate.prune_net(adata_sp)
ssgate.Stats_Nbrs_Net(adata_sp)

In [ ]:
# After the nearest neighbor network is built, it is trained based on the above data, 
# and the integrated results obtained from the training are retained in the .obsm file of h5ad.
adata_st, adata_sp = ssgate.train(adata_st, adata_sp, 
                                    hidden_dims1 = 128, 
                                    hidden_dims2 = 128, 
                                    out_dims = 30, 
                                    cluster_update_epoch = 50, 
                                    epochs_init = 50, 
                                    n_epochs=200, 
                                    save_reconstrction=False, 
                                    sigma = 0.5, 
                                    device = "cuda:0", 
                                    feat1 = "PCA",
                                    key_added = 'ssgate_embed')

In [ ]:
# Calculate neighbors and construct UMAP graph based on integrated low-dimensional embedding
sc.pp.neighbors(adata_st, use_rep="ssgate_embed",key_added='SSGATE_neighbor')
sc.tl.umap(adata_st,neighbors_key='SSGATE_neighbor')

In [ ]:
## Save the data for Monocle3 pseudo-timing analysis as a csv file in advance
adata_st.to_df().to_csv('Monocle3_folder/expression_matrix_pca.csv')
adata_st.obs.to_csv('Monocle3_folder/metadata_pca.csv')
adata_st.var.to_csv('Monocle3_folder/gene_metadata_pca.csv')
pd.DataFrame(adata_st.obsm['X_umap']).to_csv('Monocle3_folder/umap_pca.csv')
pd.DataFrame(adata_st.obsm['ssgate_embed']).to_csv('Monocle3_folder/ssgate_embed_pca.csv')

In [ ]:
# Leiden cluster statistics and drawing UMAP and spatial graph
sc.tl.leiden(adata_st, resolution = 0.15, key_added = "ssgate_cluster")

sc.pl.umap(adata_st,color='ssgate_cluster')
sc.pl.spatial(adata_st,color='ssgate_cluster', spot_size = 1.5)

In [ ]:
# Statistics of differentially expressed genes based on clustering results
adata_st.uns['log1p'] = {'base': None}
sc.tl.rank_genes_groups(adata_st, groupby='ssgate_cluster')
result = adata_st.uns['rank_genes_groups']
celltype_list = adata_st.obs.celltype_SSMI_DGATE_leiden.cat.categories.to_list()
df_all = pd.DataFrame({})

for i in range(len(celltype_list)):
    group = celltype_list[i]
    highly_expressed_genes = result['names'][group][:100]  # Get the top 100 genes
    confidence_scores = result['scores'][group][:100]  # Get the corresponding confidence
    pval = result['pvals'][group][:100]
    logfoldchange = result['logfoldchanges'][group][:100]
    
    # Merge arrays into a DataFrame
    df = pd.DataFrame({'Group': str(group), 'Gene_name': highly_expressed_genes, 'Confidence_scores': confidence_scores, 'Pval': pval, 'logfoldchanges': logfoldchange})

    # Set the index name for the DataFrame
    df.index.name = 'Index'
    print('Cell type : ', str(group))
    # Output results
    print(df)
    df_all = pd.concat([df_all, df], ignore_index=True)
# Result Output
df_all.to_csv('SSGATE_100DEgene_SCS_MT.csv')

In [ ]:
# Performing PAGA-based pseudotime time analysis

# Preprocessing and visualization
sc.tl.draw_graph(adata_st, neighbors_key = 'SSGATE_neighbor')
sc.pl.draw_graph(adata_st, color='ssgate_cluster', legend_loc='on data', save='Force-directed_graph.png')


# Create a spatial image and save it as a 300 DPI PNG
fig, ax = plt.subplots(figsize=(12, 8))
# Adjust the font and trackball size of the PAGA graph
sc.pl.paga(
    adata_st, color='ssgate_cluster',
    fontsize=30,  edge_width_scale = 3.0,       # 设置字体大小
    node_size_scale=20, ax=ax, show=False)
plt.savefig('paga_SCS_MT_SSGATE_leiden.pdf', dpi=300, format='PDF')

In [ ]:
# Specifying pathways and gene lists
nodes_path = ['5','1', '0','3','4','2']
genes_of_interest = ["Dntt", "Rag1","Trbc1",'H2-K1',"Fcgbp","Gpx3"]


# Try different root nodes
# Combined with GO enrichment analysis, 5 is the original node of T cell development
root_cells = ['5']  # Assume these are possible root nodes
for root_cell in root_cells:
    adata_st.uns['iroot'] = np.flatnonzero(adata_st.obs['ssgate_cluster'] == root_cell)[0]
    sc.tl.dpt(adata_st)
    sc.pl.draw_graph(adata_st, color=['ssgate_cluster', 'dpt_pseudotime'], legend_loc='on data', title=['', 'pseudotime'], frameon=True, save=f'_dpt_{root_cell}.png')
    sc.pl.paga_path(adata_st, nodes=nodes_path, keys=genes_of_interest, save=f'_paga_path_{root_cell}.png')